In [5]:
import pandas as pd

In [2]:
!wget -nc "http://cap-payments.defra.gov.uk/Download/2014_All_CAP_Search_Results_Data_P14.xls"

File '2014_All_CAP_Search_Results_Data_P14.xls' already there; not retrieving.



In [1]:
!wget -nc "http://cap-payments.defra.gov.uk/Download/2015_All_CAP_Search_Results_Data_P14.xls"

--2016-06-02 12:01:45--  http://cap-payments.defra.gov.uk/Download/2015_All_CAP_Search_Results_Data_P14.xls
Resolving cap-payments.defra.gov.uk... 194.1.210.50
Connecting to cap-payments.defra.gov.uk|194.1.210.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33282560 (32M) [application/vnd.ms-excel]
Saving to: '2015_All_CAP_Search_Results_Data_P14.xls'

2015_All_CAP_Search 100%[===================>]  31.74M  1.91MB/s    in 12s     

2016-06-02 12:01:57 (2.72 MB/s) - '2015_All_CAP_Search_Results_Data_P14.xls' saved [33282560/33282560]



In [2]:
!wget -nc "http://cap-payments.defra.gov.uk/Download/2016_All_CAP_Search_Results_Data_P14.xlsx"

--2018-02-05 11:36:33--  http://cap-payments.defra.gov.uk/Download/2016_All_CAP_Search_Results_Data_P14.xlsx
Resolving cap-payments.defra.gov.uk... 194.1.210.50
Connecting to cap-payments.defra.gov.uk|194.1.210.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15767150 (15M) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘2016_All_CAP_Search_Results_Data_P14.xlsx’

2016_All_CAP_Search 100%[===================>]  15.04M  2.35MB/s    in 7.7s    

2018-02-05 11:36:41 (1.96 MB/s) - ‘2016_All_CAP_Search_Results_Data_P14.xlsx’ saved [15767150/15767150]



In [3]:
YEAR = 2016

In [7]:
excel_file = pd.ExcelFile('%s_All_CAP_Search_Results_Data_P14.xlsx' % YEAR)
excel_file.sheet_names

['DAERA', 'SGRPID', 'WG', 'RPA', 'RPA2']

In [8]:
id_vars = ['Year', 'BeneficiaryCode', 'BeneficiaryName_F201',
       'PostcodePrefix_F202B', 'TownCity_F202C',
#        'OtherEAGFTotal', 'DirectEAGFTotal', 'RuralDevelopmentTotal', 'Total',
        'PayingAgencyLink']


def get_melted_df(excel_file, sheet_name):
    df_raw = excel_file.parse(sheet_name)
    df_raw = df_raw[df_raw['Year'].notnull()]
    all_columns = list(df_raw.columns)
    value_vars = list(set(all_columns) - set(id_vars))
    df_raw = df_raw.rename(columns={x: x.strip() for x in value_vars})
    value_vars = [v.strip() for v in value_vars]
    df = pd.melt(df_raw, id_vars=id_vars, value_vars=value_vars, var_name='scheme', value_name='amount')
    df = df[df['amount'].notnull()]
    return df

df = pd.concat([get_melted_df(excel_file, sheet) for sheet in excel_file.sheet_names])
df.head()

,Year,BeneficiaryCode,BeneficiaryName_F201,PostcodePrefix_F202B,TownCity_F202C,PayingAgencyLink,scheme,amount
26142,2016,1341095.0,*******,BT78,OMAGH,DAERA,Payments to areas facing natural and other spe...,209.50
26162,2016,1352021.0,*******,BT75,FIVEMILETOWN,DAERA,Payments to areas facing natural and other spe...,374.40
26193,2016,1162663.0,*******,BT31,CASTLEWELLAN,DAERA,Payments to areas facing natural and other spe...,221.93
26215,2016,1004277.0,*******,BT34,NEWRY,DAERA,Payments to areas facing natural and other spe...,176.75
26229,2016,1108471.0,*******,BT74,ENNISKILLEN,DAERA,Payments to areas facing natural and other spe...,256.37


In [9]:
df = df.rename(columns={
    'Year': 'year',
    'BeneficiaryCode': 'recipient_id',
    'BeneficiaryName_F201': 'recipient_name',
    'PostcodePrefix_F202B': 'recipient_postcode',
    'TownCity_F202C': 'recipient_location',
    'PayingAgencyLink': 'agency',
})
df['country'] = 'GB'
df['currency'] = 'GBP'
df['year'] = df['year'].astype('int')
df['recipient_name'] = df['recipient_name'].str.strip()
df['recipient_postcode'] = df['recipient_postcode'].str.strip()
df['recipient_location'] = df['recipient_location'].str.strip()
df.set_value(df['recipient_id'].notnull(), 'recipient_name', None)
df.head()

/Users/sw/.virtualenvs/notebook36/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app


,year,recipient_id,recipient_name,recipient_postcode,recipient_location,agency,scheme,amount,country,currency
26142,2016,1341095.0,None,BT78,OMAGH,DAERA,Payments to areas facing natural and other spe...,209.50,GB,GBP
26162,2016,1352021.0,None,BT75,FIVEMILETOWN,DAERA,Payments to areas facing natural and other spe...,374.40,GB,GBP
26193,2016,1162663.0,None,BT31,CASTLEWELLAN,DAERA,Payments to areas facing natural and other spe...,221.93,GB,GBP
26215,2016,1004277.0,None,BT34,NEWRY,DAERA,Payments to areas facing natural and other spe...,176.75,GB,GBP
26229,2016,1108471.0,None,BT74,ENNISKILLEN,DAERA,Payments to areas facing natural and other spe...,256.37,GB,GBP


In [10]:
def set_recipient_id(row):
    if pd.isnull(row['recipient_id']):
        row['recipient_id'] = 'GB-%s-%s' % (row['recipient_postcode'], row['recipient_name'])
    else:
        row['recipient_id'] = 'GB-%s' % int(row['recipient_id'])
    return row

df = df.apply(set_recipient_id, axis=1)
df.head()

,year,recipient_id,recipient_name,recipient_postcode,recipient_location,agency,scheme,amount,country,currency
26142,2016,GB-1341095,None,BT78,OMAGH,DAERA,Payments to areas facing natural and other spe...,209.50,GB,GBP
26162,2016,GB-1352021,None,BT75,FIVEMILETOWN,DAERA,Payments to areas facing natural and other spe...,374.40,GB,GBP
26193,2016,GB-1162663,None,BT31,CASTLEWELLAN,DAERA,Payments to areas facing natural and other spe...,221.93,GB,GBP
26215,2016,GB-1004277,None,BT34,NEWRY,DAERA,Payments to areas facing natural and other spe...,176.75,GB,GBP
26229,2016,GB-1108471,None,BT74,ENNISKILLEN,DAERA,Payments to areas facing natural and other spe...,256.37,GB,GBP


In [11]:
len(df)

1135352

In [12]:
df = df[~df['scheme'].isin([x for x in df['scheme'].value_counts().index if x.endswith('Total')])]
df['scheme'].value_counts()

Greening: practices beneficial for climate and environment                              144673
Basic payment scheme                                                                    144317
Reimbursement of financial discipline                                                   110149
Agri-environment-climate                                                                 59572
Redistributive payment                                                                   15684
Payments to areas facing natural and other specific constraints                           9074
Investment in forest area development and improvement of the viability of forests         8974
Other direct aids                                                                         7949
Investments in physical assets                                                            5311
Payment for young farmers                                                                 4961
Agri-environment payments                         

In [13]:
df.to_csv('gb_%s.csv.gz' % YEAR, index=False, encoding='utf-8', compression='gzip')